In [ ]:
from resume_parser import ResumeParser

parser = ResumeParser()
pdf_file = '/home/deepctrl/test/resume/【CV算法工程师 _ 北京25-45K】孙艾霞 3年.pdf'

text_list = parser.reader.read(pdf_file)
segments = parser.segment(text_list)


In [ ]:
resume = parser.parse(pdf_file)

In [ ]:
resume.school

In [ ]:
segments['project_segment']

In [ ]:
from helpers import Exp_Decode, Exp_Extract
edu_decode = Exp_Decode()
edu_exp = edu_decode.descrip_extract(segments['project_segment'])
for info in edu_exp:
    print('info: ', info)

In [ ]:
edu_exp

In [ ]:
import hanlp
exp_parser = Exp_Extract(hanlp.load(hanlp.pretrained.ner.MSRA_NER_ELECTRA_SMALL_ZH))

edu_info = exp_parser.correct_eduinfo(edu_exp)
print(edu_info)

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams, LTTextBoxHorizontal, LTTextBox
from pdfminer.converter import PDFPageAggregator
from collections import defaultdict
import pandas as pd

def is_leftright(df):
    if df["x0"][df["x0"]>190].count() > 20:
        return True
    else:
        False
        
file_name = '/home/deepctrl/test/resume/【CV算法工程师 _ 北京25-45K】孙艾霞 3年.pdf'
parser = PDFParser(open(file_name,'rb'))  # 用文件对象来创建一个pdf文档分析器，PDFParser从文件中提取数据
document = PDFDocument(parser)  # 创建一个PDF文档 PDFDocument保存数据，存储文档数据结构到内存中 
if not document.is_extractable:   # 检测文件是否提供txt转换，不提供就忽视
    raise PDFTextExtractionNotAllowed
rsrcmgr = PDFResourceManager()    # PDFResourceManager：pdf 共享资源管理器,用于存储共享资源，如字体或图像。
device = PDFDevice(rsrcmgr)       # 把解析到的内容转化为你需要的东西

# BEGIN LAYOUT ANALYSIS
laparams = LAParams()     # 加入需要解析的参数   
device = PDFPageAggregator(rsrcmgr, laparams=laparams)  # 创建一个PDF设备对象
interpreter = PDFPageInterpreter(rsrcmgr, device)    # 创建一个PDF解释器对象，解析page内容
page_number = len([page for page in PDFPage.create_pages(document)])  # 循环遍历列表，每次处理一个page的内容

for page in PDFPage.create_pages(document):
    print(page.mediabox)
    page_dict =  defaultdict(list)
    interpreter.process_page(page)
    layout = device.get_result() 
    page_list = []
    for lt_obj in layout:         # 遍历每一页的每一个元素     
        if isinstance(lt_obj, LTTextBox):
            line_num = len(lt_obj.get_text().strip('\n').split('\n'))
            if line_num == 1:
                page_dict["x0"].append(lt_obj.bbox[0])
                page_dict["y0"].append(page.mediabox[3] - lt_obj.bbox[3])
                page_dict["text"].append(lt_obj.get_text().strip())
            else:
                line_height = (lt_obj.bbox[3]-lt_obj.bbox[1])/line_num
#                 print(lt_obj.get_text(), line_height, lt_obj.bbox)
                for idx, line in enumerate(lt_obj.get_text().strip('\n').split('\n')):
                    page_dict['x0'].append(lt_obj.bbox[0])
                    page_dict['y0'].append(page.mediabox[3] - lt_obj.bbox[3] + idx*line_height)
                    page_dict['text'].append(line)
    
#     page_df = pd.DataFrame(page_dict)
    page_df = pd.DataFrame(page_dict).sort_values("y0", ascending=True)
    print(page_df)
#     print(page_df["x0"][page_df["x0"]>190].count())

In [ ]:
for idx, page in enumerate(page_df.text):
    print(idx, page)

In [ ]:
info = parser.extractor.extract_all(text_list, segments)

In [ ]:
info['campus_token']

In [ ]:
from helpers import Exp_Decode, Exp_Extract, extract_skill
for text in segments["other_segment"]:
    print(extract_skill(text))